In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

In [3]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

In [5]:
linear_svm = SVC(kernel='linear', C=1)
linear_svm.fit(X_train, y_train)
y_pred_linear = linear_svm.predict(X_test)

print("Linear Kernel Accuracy:", accuracy_score(y_test, y_pred_linear))

Linear Kernel Accuracy: 0.956140350877193


In [6]:
rbf_svm = SVC(kernel='rbf', C=1, gamma='scale')
rbf_svm.fit(X_train, y_train)
y_pred_rbf = rbf_svm.predict(X_test)

print("RBF Kernel Accuracy:", accuracy_score(y_test, y_pred_rbf))

RBF Kernel Accuracy: 0.9736842105263158


In [7]:
print("Linear Kernel Report:\n", classification_report(y_test, y_pred_linear))
print("RBF Kernel Report:\n", classification_report(y_test, y_pred_rbf))

Linear Kernel Report:
               precision    recall  f1-score   support

           0       0.93      0.95      0.94        43
           1       0.97      0.96      0.96        71

    accuracy                           0.96       114
   macro avg       0.95      0.96      0.95       114
weighted avg       0.96      0.96      0.96       114

RBF Kernel Report:
               precision    recall  f1-score   support

           0       0.98      0.95      0.96        43
           1       0.97      0.99      0.98        71

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114



In [8]:
C_values = [0.1, 1, 10, 100]
gamma_values = ['scale', 0.01, 0.1, 1]

print("🔧 RBF Kernel Tuning Results:")
for c in C_values:
    for g in gamma_values:
        svm_model = SVC(kernel='rbf', C=c, gamma=g)
        svm_model.fit(X_train, y_train)
        score = accuracy_score(y_test, svm_model.predict(X_test))
        print(f"C={c}, gamma={g} → Accuracy: {score:.2f}")

🔧 RBF Kernel Tuning Results:
C=0.1, gamma=scale → Accuracy: 0.95
C=0.1, gamma=0.01 → Accuracy: 0.96
C=0.1, gamma=0.1 → Accuracy: 0.95
C=0.1, gamma=1 → Accuracy: 0.62
C=1, gamma=scale → Accuracy: 0.97
C=1, gamma=0.01 → Accuracy: 0.96
C=1, gamma=0.1 → Accuracy: 0.96
C=1, gamma=1 → Accuracy: 0.63
C=10, gamma=scale → Accuracy: 0.97
C=10, gamma=0.01 → Accuracy: 0.98
C=10, gamma=0.1 → Accuracy: 0.96
C=10, gamma=1 → Accuracy: 0.63
C=100, gamma=scale → Accuracy: 0.94
C=100, gamma=0.01 → Accuracy: 0.96
C=100, gamma=0.1 → Accuracy: 0.96
C=100, gamma=1 → Accuracy: 0.63


In [9]:
best_svm = SVC(kernel='rbf', C=10, gamma=0.01)
scores = cross_val_score(best_svm, X_scaled, y, cv=5)
print("Cross-Validation Scores:", scores)
print("Average Accuracy:", scores.mean())

Cross-Validation Scores: [0.97368421 0.97368421 0.98245614 0.97368421 0.99115044]
Average Accuracy: 0.9789318428815401


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_2D = pca.fit_transform(X_scaled)

X_train2, X_test2, y_train2, y_test2 = train_test_split(X_2D, y, test_size=0.2, random_state=42)

svm_2d = SVC(kernel='rbf', C=10, gamma=0.01)
svm_2d.fit(X_train2, y_train2)

# Decision boundary plot
h = 0.02
x_min, x_max = X_2D[:, 0].min() - 1, X_2D[:, 0].max() + 1
y_min, y_max = X_2D[:, 1].min() - 1, X_2D[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
Z = svm_2d.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.figure(figsize=(10,6))
plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)
plt.scatter(X_2D[:, 0], X_2D[:, 1], c=y, cmap=plt.cm.coolwarm, edgecolors='k')
plt.title("SVM Decision Boundary (RBF, PCA 2D)")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.show()